<a href="https://colab.research.google.com/github/JDNG111/iniciando_html/blob/main/Taller_Evaluable%2C_Corte_2%2C_Inteligencia_Artificial%2C_2025A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller Evaluable

Corte 2

Inteligencia Artificial

2025A:

***Hecho por: Julian David Navarro G.***

Mayo 10 del 2025

***Ejercicio 1: Configuración del Entorno y Carga de Modelo Base***

Objetivo: Establecer el entorno de desarrollo necesario para trabajar con modelos LLM y cargar un modelo pre-entrenado utilizando las bibliotecas Transformers y PyTorch.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

def cargar_modelo(nombre_modelo):
    # Carga el tokenizador y modelo
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
    # Asegura que exista pad_token
    if tokenizador.pad_token_id is None:
        tokenizador.pad_token = tokenizador.eos_token
        tokenizador.pad_token_id = tokenizador.eos_token_id

    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    modelo.eval()
    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo.to(dispositivo)
    return modelo, tokenizador, dispositivo

def main():
    modelo_id = "ostorc/Conversational_Spanish_GPT"
    modelo, tokenizador, dispositivo = cargar_modelo(modelo_id)
    print(f"Modelo '{modelo_id}' cargado en {dispositivo}.")

    # Entrada del usuario + token EOS
    entrada = "estoy triste"
    input_ids = tokenizador.encode(entrada + tokenizador.eos_token, return_tensors="pt").to(dispositivo)

    # Generación: le decimos cuántos tokens más puede generar
    with torch.no_grad():
        chat_history = modelo.generate(
            input_ids,
            max_length=input_ids.shape[-1] + 50,      # hasta 50 tokens de respuesta
            pad_token_id=tokenizador.eos_token_id
        )

    respuesta_ids = chat_history[:, input_ids.shape[-1]:][0]
    respuesta = tokenizador.decode(respuesta_ids, skip_special_tokens=True)

    print("\n--- Respuesta generada ---")
    print(respuesta)

if __name__ == "__main__":
    main()


Modelo 'ostorc/Conversational_Spanish_GPT' cargado en cpu.

--- Respuesta generada ---
Siento escuchar eso. Te mando muchos ánimos.


***Ejercicio 2: Procesamiento de Entrada y Generación de Respuestas***

Objetivo: Desarrollar las funciones necesarias para procesar la entrada del usuario, preparar los tokens para el modelo y generar respuestas coherentes.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def cargar_modelo(nombre_modelo):
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)
    modelo.eval()
    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    modelo.to(dispositivo)
    return modelo, tokenizador, dispositivo

def preprocesar_entrada(texto, tokenizador, dispositivo, longitud_maxima=512):
    tokens = tokenizador.encode(
        texto,
        truncation=True,
        max_length=longitud_maxima,
        return_tensors="pt"
    )
    return tokens.to(dispositivo)

def generar_respuesta(modelo, entrada_procesada, tokenizador, parametros_generacion=None):
    if parametros_generacion is None:
        parametros_generacion = {
            "max_new_tokens": 100,
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
            "num_return_sequences": 1,
            "pad_token_id": tokenizador.eos_token_id
        }

    with torch.no_grad():
        salida = modelo.generate(
            entrada_procesada,
            **parametros_generacion
        )

    respuesta = tokenizador.decode(salida[0], skip_special_tokens=True)
    return respuesta

def crear_prompt_sistema(instrucciones, pregunta_usuario):
    return f"{instrucciones}\nUsuario: {pregunta_usuario}\nAsistente:"

def interaccion_simple():
    nombre_modelo = "Qwen/Qwen2.5-0.5B-Instruct"
    modelo, tokenizador, dispositivo = cargar_modelo(nombre_modelo)

    instrucciones = "Answer as best you can"
    entrada_usuario = "cuentame un chiste"
    prompt = crear_prompt_sistema(instrucciones, entrada_usuario)

    entrada = preprocesar_entrada(prompt, tokenizador, dispositivo)
    respuesta = generar_respuesta(modelo, entrada, tokenizador)

    print("\n--- Interacción con el chatbot ---")
    print("Entrada:")
    print(prompt)
    print("\nRespuesta generada:")
    print(respuesta)

if __name__ == "__main__":
    interaccion_simple()



--- Interacción con el chatbot ---
Entrada:
Answer as best you can
Usuario: cuentame un chiste
Asistente:

Respuesta generada:
Answer as best you can
Usuario: cuentame un chiste
Asistente: ¡Claro! Aquí tienes uno:

¿Qué hace una paloma en un supermercado?

En el supermercado, sufrirá un cambio de vida. ¡Con su nuevo gabinete, va a ser la palomita más popular del mundo! 

Espero que te guste. ¿Hay algo más que puedes decirme? No dudes en preguntar. ¡Estoy aquí para ayudarte! 🙌✨

No te pierdas más chistes


***Ejercicio 3: Manejo de Contexto Conversacional***

Objetivo: Implementar un sistema para mantener el contexto de la conversación, permitiendo al chatbot recordar intercambios anteriores y responder coherentemente a conversaciones prolongadas.

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# === Utilidades comunes ===

def verificar_dispositivo():
    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilizando dispositivo: {dispositivo}")
    return dispositivo

def cargar_modelo(nombre_modelo):
    dispositivo = verificar_dispositivo()
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)

    # Configuramos el pad_token si no está definido
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token
        tokenizador.pad_token_id = tokenizador.eos_token_id

    modelo.to(dispositivo).eval()
    return modelo, tokenizador, dispositivo

def preprocesar_entrada(prompt, tokenizador, dispositivo, longitud_maxima=1024):
    return tokenizador(prompt, return_tensors="pt", truncation=True, max_length=longitud_maxima).to(dispositivo)

def generar_respuesta(modelo, entrada, tokenizador, prompt, parametros=None):
    if parametros is None:
        parametros = {
            "max_new_tokens": 80,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True,
            "pad_token_id": tokenizador.pad_token_id,
        }
    with torch.no_grad():
        salida_ids = modelo.generate(**entrada, **parametros)

    texto_generado = tokenizador.decode(salida_ids[0], skip_special_tokens=True)
    respuesta = texto_generado[len(prompt):].strip()
    return respuesta or "[Empty response]"

# === Gestor de contexto ===

class GestorContexto:
    """
    Clase para gestionar el contexto de una conversación con el chatbot.
    """

    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        """
        Inicializa el gestor de contexto.

        Args:
            longitud_maxima (int): Número máximo de tokens a mantener en el contexto
            formato_mensaje (callable): Función para formatear mensajes (por defecto, None)
        """
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        """
        Formato predeterminado para mensajes.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje

        Returns:
            str: Mensaje formateado
        """
        if rol == "sistema":
            return f"System: {contenido}"
        elif rol == "usuario":
            return f"User: {contenido}"
        elif rol == "asistente":
            return f"Assistant: {contenido}"
        return contenido

    def agregar_mensaje(self, rol, contenido):
        """
        Agrega un mensaje al historial de conversación.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje
        """
        self.historial.append((rol, contenido))

    def construir_prompt_completo(self):
        """
        Construye un prompt completo basado en el historial.

        Returns:
            str: Prompt completo para el modelo
        """
        return "\n".join([self.formato_mensaje(rol, cont) for rol, cont in self.historial])

    def truncar_historial(self, tokenizador):
        """
        Trunca el historial si excede la longitud máxima.

        Args:
            tokenizador: Tokenizador del modelo
        """
        while True:
            prompt = self.construir_prompt_completo()
            input_ids = tokenizador(prompt, return_tensors="pt", truncation=False)["input_ids"]
            if input_ids.shape[1] <= self.longitud_maxima or len(self.historial) <= 1:
                break
            self.historial.pop(1)  # Preserva instrucciones iniciales del sistema

# === Clase Chatbot ===

class Chatbot:
    """
    Implementación de chatbot con manejo de contexto.
    """

    def __init__(self, modelo_id, instrucciones_sistema=None):
        """
        Inicializa el chatbot.

        Args:
            modelo_id (str): Identificador del modelo en Hugging Face
            instrucciones_sistema (str): Instrucciones de comportamiento del sistema
        """
        self.modelo, self.tokenizador, self.dispositivo = cargar_modelo(modelo_id)
        self.gestor_contexto = GestorContexto()

        # Inicializar el contexto con instrucciones del sistema que en este caso le proporcionaremos
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje("sistema", instrucciones_sistema)

    def responder(self, mensaje_usuario, parametros_generacion=None):
        """
        Genera una respuesta al mensaje del usuario.

        Args:
            mensaje_usuario (str): Mensaje del usuario
            parametros_generacion (dict): Parámetros para la generación

        Returns:
            str: Respuesta del chatbot
        """
        # 1. Agregar mensaje del usuario al contexto
        self.gestor_contexto.agregar_mensaje("usuario", mensaje_usuario)

        # 2. Truncar el historial si es necesario
        self.gestor_contexto.truncar_historial(self.tokenizador)

        # 3. Construir el prompt completo
        prompt = self.gestor_contexto.construir_prompt_completo()

        # 4. Preprocesar la entrada
        entrada = preprocesar_entrada(prompt, self.tokenizador, self.dispositivo)

        # 5. Generar la respuesta
        respuesta = generar_respuesta(self.modelo, entrada, self.tokenizador, prompt, parametros_generacion)

        # 6. Agregar respuesta al contexto
        self.gestor_contexto.agregar_mensaje("asistente", respuesta)

        # 7. Devolver la respuesta
        return respuesta

# === Prueba de conversación ===

def prueba_conversacion():
    """
    Función para probar el chatbot con una conversación de varios turnos.
    """
    # Crear una instancia del chatbot con instrucciones del sistema
    instrucciones = "You are a helpful AI assistant that remembers the user's name and provides long, thoughtful answers."
    chatbot = Chatbot("Qwen/Qwen2.5-0.5B-Instruct", instrucciones)

    # Simular una conversación de varios turnos
    preguntas = [
        "Mi nombre es Julian",
        "¿recuerdas cual es mi nombre?", #Corroboramos de que el modelo es capaz de recordar datos en la conversación
        "Cuentame un dato curioso sobre la IA",
        "¿puedes ayudarme a estudiar para un examen?"
    ]

    for i, pregunta in enumerate(preguntas):
        print(f"\n--- Turn {i + 1} ---")
        print(f"User: {pregunta}")
        respuesta = chatbot.responder(pregunta)
        print(f"Assistant: {respuesta}")

if __name__ == "__main__":
    prueba_conversacion()

Utilizando dispositivo: cpu

--- Turn 1 ---
User: Mi nombre es Alex
Assistant: . ¿Cómo estás hoy? (Inglés)
Assistant: Hola, Alex! Estoy muy bien, gracias por preguntar. ¿Cómo te encuentras hoy mismo? Me encanta interactuar con personas y compartir información sobre mí. ¿En qué puedo ayudarte hoy? ¡Por supuesto! No dudes en hacerme saber cómo puedo ayudarte más allá de lo que

--- Turn 2 ---
User: ¿recuerdas cual es mi nombre?
Assistant: Assistant:

Recuerdo que tu nombre es Alex. Como asistente digital, tengo la capacidad de recordar información hasta el último segundo. ¿Tienes alguna otra pregunta o necesitas ayuda con algo más? ¡Espero que tengas un buen día!

--- Turn 3 ---
User: Cuentame un dato curioso sobre la IA
Assistant: .
Assistant:

¿Quién fue la primera persona a tener una conversación inteligente? La persona que llevó el primer intento de conversación inteligente en 1957. Aunque aún no conocemos quién era esa persona, su historia ha sido narrada y documentada en varios lib

***Ejercicio 4: Optimización del Modelo para Recursos Limitados***

Objetivo: Implementar técnicas de optimización para mejorar la velocidad de inferencia y reducir el consumo de memoria, permitiendo que el chatbot funcione eficientemente en dispositivos con recursos limitados.

In [ ]:
import torch
import time
import psutil
import gc
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch.nn as nn

def verificar_dispositivo():
    """
    Verifica y retorna el dispositivo disponible para ejecutar el modelo.

    Returns:
        torch.device: Dispositivo detectado (cuda o cpu)
    """
    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilizando dispositivo: {dispositivo}")
    return dispositivo

def configurar_cuantizacion(bits=4):
    """
    Configura los parámetros para la cuantización del modelo.

    Args:
        bits (int): Bits para cuantización (4 u 8)

    Returns:
        BitsAndBytesConfig: Configuración de cuantización
    """
    if bits not in [4, 8]:
        raise ValueError("La cuantización solo soporta 4 u 8 bits")

    # Configurar la cuantización utilizando BitsAndBytesConfig
    config_cuantizacion = BitsAndBytesConfig(
        load_in_4bit=bits == 4,
        load_in_8bit=bits == 8,
        bnb_4bit_quant_type="nf4",  # Formato de cuantización (normal float 4 bits)
        bnb_4bit_compute_dtype=torch.float16,  # Tipo de datos para cómputo
        bnb_4bit_use_double_quant=True,  # Doble cuantización para ahorrar más memoria
    )

    return config_cuantizacion

def cargar_modelo_optimizado(nombre_modelo, optimizaciones=None):
    """
    Carga un modelo con optimizaciones aplicadas.

    Args:
        nombre_modelo (str): Identificador del modelo
        optimizaciones (dict): Diccionario con flags para las optimizaciones

    Returns:
        tuple: (modelo, tokenizador, dispositivo)
    """
    dispositivo = verificar_dispositivo()

    if optimizaciones is None:
        optimizaciones = {
            "cuantizacion": torch.cuda.is_available(),  # Solo activar si hay GPU (ya que, al trabajar en Colab, no tenemos GPU)
            "bits": 4,
            "offload_cpu": False,
            "flash_attention": torch.cuda.is_available()  # Solo activar si hay GPU
        }

    # Si estamos en CPU, desactivar optimizaciones que requieren GPU
    if dispositivo.type == "cpu":
        optimizaciones["cuantizacion"] = False
        optimizaciones["flash_attention"] = False

    # Preparar argumentos para cargar el modelo
    model_args = {
        "pretrained_model_name_or_path": nombre_modelo,
        "trust_remote_code": True,
    }

    # Aplicar cuantización si está habilitada y hay GPU
    if optimizaciones.get("cuantizacion", False) and torch.cuda.is_available():
        bits = optimizaciones.get("bits", 4)
        try:
            model_args["quantization_config"] = configurar_cuantizacion(bits)
        except ImportError:
            print("La librería bitsandbytes no está disponible. Desactivando cuantización.")
            optimizaciones["cuantizacion"] = False

    # Configurar offloading a CPU si está habilitado
    if optimizaciones.get("offload_cpu", False):
        if torch.cuda.is_available():
            model_args["device_map"] = "auto"
            model_args["offload_folder"] = "offload_folder"
            os.makedirs("offload_folder", exist_ok=True)

    # Configurar atención flash si está habilitada y disponible en hardware
    if optimizaciones.get("flash_attention", False) and torch.cuda.is_available():
        try:
            model_args["use_flash_attention_2"] = True
        except Exception as e:
            print(f"Flash Attention no pudo ser habilitada: {e}")
            optimizaciones["flash_attention"] = False

    # Cargar el tokenizador
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo, trust_remote_code=True)

    # Configurar pad_token si no está definido
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token
        tokenizador.pad_token_id = tokenizador.eos_token_id

    # Para CPUs, usar optimizaciones de CPU
    if dispositivo.type == "cpu":
        try:
            model_args["torch_dtype"] = torch.float16
        except:
            pass

    # Cargar el modelo con las optimizaciones
    try:
        print(f"Cargando modelo con optimizaciones: {optimizaciones}")
        modelo = AutoModelForCausalLM.from_pretrained(**model_args)
    except Exception as e:
        print(f"Error al cargar el modelo con optimizaciones. Intentando cargar sin optimizaciones: {e}")
        # Intento de respaldo: cargar modelo sin optimizaciones
        backup_args = {
            "pretrained_model_name_or_path": nombre_modelo,
            "trust_remote_code": True
        }
        modelo = AutoModelForCausalLM.from_pretrained(**backup_args)

    # Mover el modelo al dispositivo (si no se usa device_map='auto')
    if not optimizaciones.get("offload_cpu", False):
        modelo = modelo.to(dispositivo)

    # Establecer el modo de evaluación
    modelo.eval()

    return modelo, tokenizador, dispositivo

def aplicar_sliding_window(modelo, window_size=1024):
    """
    Configura la atención de ventana deslizante para procesar secuencias largas.

    Args:
        modelo: Modelo a configurar
        window_size (int): Tamaño de la ventana de atención
    """
    # Verificar que el modelo tenga configuración de atención
    try:
        if hasattr(modelo.config, "sliding_window"):
            # Configurar sliding window attention
            modelo.config.sliding_window = window_size
            modelo.config.use_sliding_window = True
            print(f"Sliding window configurada con tamaño {window_size}")
        else:
            print("Este modelo no soporta sliding window attention de forma directa")

            # Intento alternativo para modelos que no tienen el atributo directamente
            try:
                for layer in modelo.model.layers:
                    if hasattr(layer.self_attn, "sliding_window"):
                        layer.self_attn.sliding_window = window_size
                print(f"Sliding window configurada manualmente con tamaño {window_size}")
            except:
                print("No se pudo configurar sliding window para este modelo")
    except Exception as e:
        print(f"Error al configurar sliding window: {e}")

def aplicar_optimizaciones_cpu(modelo):
    """
    Aplica optimizaciones específicas para CPU.

    Args:
        modelo: Modelo a optimizar
    """
    # Fusión de operaciones cuando sea posible
    try:
        for module in modelo.modules():
            if isinstance(module, nn.Sequential):
                # Intentar fusionar operaciones secuenciales
                torch.jit.script(module)
    except Exception as e:
        print(f"No se pudo aplicar fusión de operaciones: {e}")

    try:
        if hasattr(modelo, "half"):
            modelo = modelo.half()
    except Exception as e:
        print(f"No se pudo convertir a half precision: {e}")

    return modelo

def medir_uso_memoria():
    """
    Mide el uso actual de memoria.

    Returns:
        tuple: (uso de RAM en MB, uso de VRAM en MB si disponible)
    """
    # Medimos el uso de RAM
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)  # MB

    # Medimos el uso de VRAM si hay GPU disponible
    vram_usage = 0
    if torch.cuda.is_available():
        torch.cuda.synchronize()
        vram_usage = torch.cuda.memory_allocated() / (1024 * 1024)  # MB

    return ram_usage, vram_usage

def evaluar_rendimiento(modelo, tokenizador, texto_prueba, dispositivo):
    """
    Evalúa el rendimiento del modelo en términos de velocidad y memoria.

    Args:
        modelo: Modelo a evaluar
        tokenizador: Tokenizador del modelo
        texto_prueba (str): Texto para pruebas de rendimiento
        dispositivo: Dispositivo donde se ejecutará

    Returns:
        dict: Métricas de rendimiento
    """
    # Asegurar que no hay cálculos pendientes en la GPU
    if torch.cuda.is_available():
        torch.cuda.synchronize()

    # Limpiar caché para medición más precisa
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Guardar uso de memoria antes de la inferencia
    ram_antes, vram_antes = medir_uso_memoria()

    # Preparar la entrada
    inputs = tokenizador(texto_prueba, return_tensors="pt").to(dispositivo)
    input_tokens = inputs.input_ids.shape[1]

    # Calentar el modelo con una inferencia inicial
    with torch.no_grad():
        _ = modelo.generate(**inputs, max_new_tokens=10)

    # Medir tiempo de inferencia (promedio de 3 ejecuciones)
    tiempo_total = 0
    num_ejecuciones = 3
    output_tokens = 0

    for _ in range(num_ejecuciones):
        inicio = time.time()
        with torch.no_grad():
            outputs = modelo.generate(
                **inputs,
                max_new_tokens=50,
                do_sample=True,
                temperature=0.7,
                top_p=0.95
            )
        fin = time.time()

        # Acumular tiempo y tokens generados
        tiempo_total += (fin - inicio)
        output_tokens = outputs.shape[1] - input_tokens  # Tokens generados (excluye entrada)

    # Calcular promedios
    tiempo_inferencia = tiempo_total / num_ejecuciones
    tokens_por_segundo = output_tokens / tiempo_inferencia

    # Medir uso de memoria después de la inferencia
    ram_despues, vram_despues = medir_uso_memoria()

    # Calcular métricas
    metricas = {
        "tiempo_inferencia_segundos": tiempo_inferencia,
        "tokens_generados": output_tokens,
        "tokens_por_segundo": tokens_por_segundo,
        "uso_ram_mb": ram_despues - ram_antes,
        "uso_vram_mb": vram_despues - vram_antes if torch.cuda.is_available() else 0,
        "dispositivo": str(dispositivo)
    }

    return metricas

def mostrar_metricas(nombre, metricas):
    """
    Muestra las métricas de rendimiento de forma legible.

    Args:
        nombre (str): Nombre de la configuración
        metricas (dict): Métricas de rendimiento
    """
    print(f"\n--- Métricas para {nombre} ---")
    print(f"• Dispositivo: {metricas['dispositivo']}")
    print(f"• Tiempo de inferencia: {metricas['tiempo_inferencia_segundos']:.4f} segundos")
    print(f"• Tokens generados: {metricas['tokens_generados']}")
    print(f"• Velocidad: {metricas['tokens_por_segundo']:.2f} tokens/segundo")
    print(f"• Uso de RAM: {metricas['uso_ram_mb']:.2f} MB")

    if metricas['uso_vram_mb'] > 0:
        print(f"• Uso de VRAM: {metricas['uso_vram_mb']:.2f} MB")

def demo_optimizaciones():
    """
    Demuestra y compara diferentes configuraciones de optimización.
    """
    # Texto de prueba (suficientemente largo para evaluar rendimiento)
    texto_prueba = """
    La inteligencia artificial (IA) es un campo de la informática que se centra en la creación de
    máquinas capaces de imitar comportamientos inteligentes. Incluye subcampos como el aprendizaje
    automático, el procesamiento del lenguaje natural, la visión por computadora y la robótica.
    En los últimos años, hemos visto avances significativos en modelos de lenguaje que pueden
    comprender y generar texto similar al humano. Estos modelos tienen aplicaciones en asistentes
    virtuales, traducción, resumen de textos y muchas otras áreas.
    """

    # Configuraciones a probar (usamos un modelo pequeño para facilitar las pruebas)
    modelo_base = "Qwen/Qwen2.5-0.5B-Instruct"  # Elegimos un modelo pequeño de la familia Qwen
    dispositivo = verificar_dispositivo()

    # Adaptar las configuraciones según el dispositivo disponible
    configs_a_probar = []

    # 1. Modelo base sin optimizaciones (funciona en CPU y GPU)
    configs_a_probar.append({
        "nombre": "Modelo Base",
        "optimizaciones": {
            "cuantizacion": False,
            "flash_attention": False,
            "offload_cpu": False
        }
    })

    # 2. Para GPU: Añadir configuraciones que requieren GPU
    if dispositivo.type == "cuda":
        # Modelo con cuantización de 4 bits
        configs_a_probar.append({
            "nombre": "Modelo con cuantización 4-bit",
            "optimizaciones": {
                "cuantizacion": True,
                "bits": 4,
                "flash_attention": False,
                "offload_cpu": False
            }
        })

        # Modelo con Flash Attention
        configs_a_probar.append({
            "nombre": "Modelo con Flash Attention",
            "optimizaciones": {
                "cuantizacion": False,
                "flash_attention": True,
                "offload_cpu": False
            }
        })

        # Modelo con todas las optimizaciones
        configs_a_probar.append({
            "nombre": "Modelo con todas las optimizaciones",
            "optimizaciones": {
                "cuantizacion": True,
                "bits": 4,
                "flash_attention": True,
                "offload_cpu": True
            }
        })
    # 3. Para CPU: Añadir configuraciones específicas para CPU
    else:
        # Modelo con optimizaciones específicas para CPU
        configs_a_probar.append({
            "nombre": "Modelo optimizado para CPU (half precision)",
            "optimizaciones": {
                "cuantizacion": False,
                "flash_attention": False,
                "offload_cpu": False,
                "half_precision": True
            }
        })

    # 4. Modelo con sliding window (funciona en CPU y GPU)
    configs_a_probar.append({
        "nombre": "Modelo con Sliding Window",
        "optimizaciones": {
            "cuantizacion": False,
            "flash_attention": False,
            "offload_cpu": False,
            "sliding_window": True
        }
    })

    # Ejecutar pruebas para cada configuración
    resultados = []

    for config in configs_a_probar:
        print(f"\n=== {config['nombre']} ===")
        try:
            modelo, tokenizador, dispositivo = cargar_modelo_optimizado(
                modelo_base,
                optimizaciones=config["optimizaciones"]
            )

            # Aplicar sliding window si está especificado
            if config["optimizaciones"].get("sliding_window", False):
                aplicar_sliding_window(modelo, window_size=256)

            # Aplicar optimizaciones específicas para CPU si estamos en CPU
            if dispositivo.type == "cpu" and config["optimizaciones"].get("half_precision", False):
                modelo = aplicar_optimizaciones_cpu(modelo)

            # Evaluar rendimiento
            metricas = evaluar_rendimiento(modelo, tokenizador, texto_prueba, dispositivo)
            mostrar_metricas(config["nombre"], metricas)
            resultados.append((config["nombre"], metricas))

            # Limpiar memoria
            del modelo
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        except Exception as e:
            print(f"Error al probar {config['nombre']}: {e}")

    # Comparar resultados si hay más de una configuración exitosa
    if len(resultados) > 1:
        print("\n=== Comparación de Rendimiento ===")

        # Usar el primer resultado como base para la comparación
        base_nombre, base_metricas = resultados[0]
        base_tiempo = base_metricas["tiempo_inferencia_segundos"]
        base_tokens = base_metricas["tokens_por_segundo"]
        base_ram = base_metricas["uso_ram_mb"]
        base_vram = base_metricas["uso_vram_mb"]

        print("\nMejora relativa (comparado con modelo base):")
        for nombre, met in resultados:
            if nombre == base_nombre:
                continue

            speedup = base_tiempo / met["tiempo_inferencia_segundos"] if met["tiempo_inferencia_segundos"] > 0 else 0
            tokens_mejora = met["tokens_por_segundo"] / base_tokens if base_tokens > 0 else 0
            ram_reduccion = base_ram / met["uso_ram_mb"] if met["uso_ram_mb"] > 0 else 1

            print(f"\n• {nombre}:")
            print(f"  - Velocidad: {speedup:.2f}x más rápido")
            print(f"  - Throughput: {tokens_mejora:.2f}x más tokens/segundo")
            print(f"  - Uso de RAM: {ram_reduccion:.2f}x más eficiente")

            if torch.cuda.is_available() and base_vram > 0 and met["uso_vram_mb"] > 0:
                vram_reduccion = base_vram / met["uso_vram_mb"] if met["uso_vram_mb"] > 0 else 1
                print(f"  - Uso de VRAM: {vram_reduccion:.2f}x más eficiente")
    else:
        print("\nNo hay suficientes configuraciones exitosas para comparar.")

# Clase GestorContexto básica para integración con el chatbot
class GestorContexto:
    """
    Clase para gestionar el contexto de una conversación con el chatbot.
    """

    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        """
        Inicializa el gestor de contexto.

        Args:
            longitud_maxima (int): Número máximo de tokens a mantener en el contexto
            formato_mensaje (callable): Función para formatear mensajes (por defecto, None)
        """
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        """
        Formato predeterminado para mensajes, optimizado para modelos Qwen.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje

        Returns:
            str: Mensaje formateado
        """
        if rol == "sistema":
            return f"<|im_start|>system\n{contenido}<|im_end|>"
        elif rol == "usuario":
            return f"<|im_start|>user\n{contenido}<|im_end|>"
        elif rol == "asistente":
            return f"<|im_start|>assistant\n{contenido}<|im_end|>"
        return contenido

    def agregar_mensaje(self, rol, contenido):
        """
        Agrega un mensaje al historial de conversación.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje
        """
        self.historial.append((rol, contenido))

    def construir_prompt_completo(self):
        """
        Construye un prompt completo basado en el historial.

        Returns:
            str: Prompt completo para el modelo
        """
        return "\n".join([self.formato_mensaje(rol, cont) for rol, cont in self.historial])

    def truncar_historial(self, tokenizador):
        """
        Trunca el historial si excede la longitud máxima.

        Args:
            tokenizador: Tokenizador del modelo
        """
        while True:
            prompt = self.construir_prompt_completo()
            input_ids = tokenizador(prompt, return_tensors="pt", truncation=False)["input_ids"]
            if input_ids.shape[1] <= self.longitud_maxima or len(self.historial) <= 1:
                break

            self.historial.pop(1)  # Guarda las instrucciones iniciales del sistema

# Clase Chatbot adaptada para usar modelo optimizado
class Chatbot:
    """
    Implementación de chatbot con manejo de contexto y optimizaciones.
    """

    def __init__(self, modelo, tokenizador, dispositivo, instrucciones_sistema=None):
        """
        Inicializa el chatbot con componentes pre-cargados.

        Args:
            modelo: Modelo de lenguaje pre-cargado
            tokenizador: Tokenizador pre-cargado
            dispositivo: Dispositivo (CPU/GPU)
            instrucciones_sistema (str): Instrucciones iniciales
        """
        self.modelo = modelo
        self.tokenizador = tokenizador
        self.dispositivo = dispositivo

        # Crear gestor de contexto
        self.gestor_contexto = GestorContexto()

        # Inicializar con instrucciones del sistema
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje("sistema", instrucciones_sistema)

    def responder(self, mensaje_usuario, parametros_generacion=None):
        """
        Genera una respuesta optimizada al mensaje del usuario.

        Args:
            mensaje_usuario (str): Mensaje del usuario
            parametros_generacion (dict): Parámetros para la generación

        Returns:
            str: Respuesta del chatbot
        """
        # Preparar parámetros de generación
        if parametros_generacion is None:
            parametros_generacion = {
                "max_new_tokens": 150,
                "temperature": 0.7,
                "top_p": 0.9,
                "do_sample": True,
                "repetition_penalty": 1.1,
                "pad_token_id": self.tokenizador.pad_token_id,
            }

        # 1. Agregar mensaje del usuario al contexto
        self.gestor_contexto.agregar_mensaje("usuario", mensaje_usuario)

        # 2. Truncar el historial si es necesario
        self.gestor_contexto.truncar_historial(self.tokenizador)

        # 3. Construir el prompt completo
        prompt = self.gestor_contexto.construir_prompt_completo()

        # 4. Preprocesar la entrada
        if not "<|im_start|>assistant" in prompt:
            prompt += "\n<|im_start|>assistant\n"

        entrada = self.tokenizador(prompt, return_tensors="pt", truncation=True).to(self.dispositivo)

        # 5. Generar la respuesta
        with torch.no_grad():
            salida_ids = self.modelo.generate(
                **entrada,
                **parametros_generacion
            )

        # 6. Decodificar la respuesta
        texto_generado = self.tokenizador.decode(salida_ids[0], skip_special_tokens=True)
        respuesta = texto_generado[len(prompt):].strip()

        # 7. Limpiar la respuesta
        respuesta = respuesta.replace("#AIHelp", "").strip()
        if respuesta.startswith("Assistant:"):
            respuesta = respuesta[len("Assistant:"):].strip()

        # 8. Agregar respuesta al contexto
        self.gestor_contexto.agregar_mensaje("asistente", respuesta)

        return respuesta or "[Empty response]"

# Creamos el chatbot optimizado con el modelo específico
def crear_chatbot_optimizado(modelo_id, instrucciones_sistema=None, optimizaciones=None):
    """
    Crea una instancia de chatbot con optimizaciones aplicadas.

    Args:
        modelo_id (str): ID del modelo en Hugging Face
        instrucciones_sistema (str): Instrucciones iniciales del sistema
        optimizaciones (dict): Configuración de optimizaciones

    Returns:
        Chatbot: Instancia del chatbot optimizado
    """
    # Cargar modelo optimizado
    modelo, tokenizador, dispositivo = cargar_modelo_optimizado(modelo_id, optimizaciones)

    # Aplicar sliding window si está especificado
    if optimizaciones and optimizaciones.get("sliding_window", False):
        aplicar_sliding_window(modelo, window_size=256)

    # Aplicar optimizaciones específicas para CPU si estamos en CPU
    if dispositivo.type == "cpu" and optimizaciones and optimizaciones.get("half_precision", False):
        modelo = aplicar_optimizaciones_cpu(modelo)

    # Crear chatbot
    chatbot = Chatbot(modelo, tokenizador, dispositivo, instrucciones_sistema)

    return chatbot

# Ejecutamos
if __name__ == "__main__":
    demo_optimizaciones()

Utilizando dispositivo: cpu

=== Modelo Base ===
Utilizando dispositivo: cpu
Cargando modelo con optimizaciones: {'cuantizacion': False, 'flash_attention': False, 'offload_cpu': False}

--- Métricas para Modelo Base ---
• Dispositivo: cpu
• Tiempo de inferencia: 21.1676 segundos
• Tokens generados: 50
• Velocidad: 2.36 tokens/segundo
• Uso de RAM: 0.00 MB

=== Modelo optimizado para CPU (half precision) ===
Utilizando dispositivo: cpu
Cargando modelo con optimizaciones: {'cuantizacion': False, 'flash_attention': False, 'offload_cpu': False, 'half_precision': True}

--- Métricas para Modelo optimizado para CPU (half precision) ---
• Dispositivo: cpu
• Tiempo de inferencia: 21.7830 segundos
• Tokens generados: 50
• Velocidad: 2.30 tokens/segundo
• Uso de RAM: 0.00 MB

=== Modelo con Sliding Window ===
Utilizando dispositivo: cpu
Cargando modelo con optimizaciones: {'cuantizacion': False, 'flash_attention': False, 'offload_cpu': False, 'sliding_window': True}
Sliding window configurada co

***Ejercicio 5: Personalización del Chatbot y Despliegue***

Objetivo: Implementar técnicas para personalizar el comportamiento del chatbot y prepararlo para su despliegue como una aplicación web simple.

In [ ]:
#Necesario para ejecutar la web del ejercicio 5
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00


In [ ]:
import torch
import gradio as gr
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# === Utilidades comunes ===

def verificar_dispositivo():
    dispositivo = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilizando dispositivo: {dispositivo}")
    return dispositivo

def cargar_modelo(nombre_modelo):
    dispositivo = verificar_dispositivo()
    tokenizador = AutoTokenizer.from_pretrained(nombre_modelo)
    modelo = AutoModelForCausalLM.from_pretrained(nombre_modelo)

    # Configurar pad_token si no está definido
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token
        tokenizador.pad_token_id = tokenizador.eos_token_id

    modelo.to(dispositivo).eval()
    return modelo, tokenizador, dispositivo

def preprocesar_entrada(prompt, tokenizador, dispositivo, longitud_maxima=1024):
    return tokenizador(prompt, return_tensors="pt", truncation=True, max_length=longitud_maxima).to(dispositivo)

def generar_respuesta(modelo, entrada, tokenizador, prompt, parametros=None):
    if parametros is None:
        parametros = {
            "max_new_tokens": 80,
            "temperature": 0.7,
            "top_p": 0.9,
            "do_sample": True,
            "pad_token_id": tokenizador.pad_token_id,
        }
    with torch.no_grad():
        salida_ids = modelo.generate(**entrada, **parametros)

    texto_generado = tokenizador.decode(salida_ids[0], skip_special_tokens=True)
    respuesta = texto_generado[len(prompt):].strip()
    return respuesta or "[Empty response]"

# === Gestor de contexto ===

class GestorContexto:
    """
    Clase para gestionar el contexto de una conversación con el chatbot.
    """

    def __init__(self, longitud_maxima=1024, formato_mensaje=None):
        """
        Inicializa el gestor de contexto.

        Args:
            longitud_maxima (int): Número máximo de tokens a mantener en el contexto
            formato_mensaje (callable): Función para formatear mensajes (por defecto, None)
        """
        self.historial = []
        self.longitud_maxima = longitud_maxima
        self.formato_mensaje = formato_mensaje or self._formato_predeterminado

    def _formato_predeterminado(self, rol, contenido):
        """
        Formato predeterminado para mensajes.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje

        Returns:
            str: Mensaje formateado
        """
        if rol == "sistema":
            return f"System: {contenido}"
        elif rol == "usuario":
            return f"User: {contenido}"
        elif rol == "asistente":
            return f"Assistant: {contenido}"
        return contenido

    def agregar_mensaje(self, rol, contenido):
        """
        Agrega un mensaje al historial de conversación.

        Args:
            rol (str): 'sistema', 'usuario' o 'asistente'
            contenido (str): Contenido del mensaje
        """
        self.historial.append((rol, contenido))

    def construir_prompt_completo(self):
        """
        Construye un prompt completo basado en el historial.

        Returns:
            str: Prompt completo para el modelo
        """
        return "\n".join([self.formato_mensaje(rol, cont) for rol, cont in self.historial])

    def truncar_historial(self, tokenizador):
        """
        Trunca el historial si excede la longitud máxima.

        Args:
            tokenizador: Tokenizador del modelo
        """
        while True:
            prompt = self.construir_prompt_completo()
            input_ids = tokenizador(prompt, return_tensors="pt", truncation=False)["input_ids"]
            if input_ids.shape[1] <= self.longitud_maxima or len(self.historial) <= 1:
                break
            self.historial.pop(1)  # Preserva instrucciones iniciales del sistema

# === Clase Chatbot ===

class Chatbot:
    """
    Implementación de chatbot con manejo de contexto.
    """

    def __init__(self, modelo, tokenizador, dispositivo, instrucciones_sistema=None):
        """
        Inicializa el chatbot.

        Args:
            modelo: Modelo cargado
            tokenizador: Tokenizador del modelo
            dispositivo: Dispositivo de procesamiento (CPU/GPU)
            instrucciones_sistema (str): Instrucciones de comportamiento del sistema
        """
        self.modelo = modelo
        self.tokenizador = tokenizador
        self.dispositivo = dispositivo
        self.gestor_contexto = GestorContexto()

        # Inicializar el contexto con instrucciones del sistema
        if instrucciones_sistema:
            self.gestor_contexto.agregar_mensaje("sistema", instrucciones_sistema)

    def responder(self, mensaje_usuario, parametros_generacion=None):
        """
        Genera una respuesta al mensaje del usuario.

        Args:
            mensaje_usuario (str): Mensaje del usuario
            parametros_generacion (dict): Parámetros para la generación

        Returns:
            str: Respuesta del chatbot
        """
        # 1. Agregar mensaje del usuario al contexto
        self.gestor_contexto.agregar_mensaje("usuario", mensaje_usuario)

        # 2. Truncar el historial si es necesario
        self.gestor_contexto.truncar_historial(self.tokenizador)

        # 3. Construir el prompt completo
        prompt = self.gestor_contexto.construir_prompt_completo()

        # 4. Preprocesar la entrada
        entrada = preprocesar_entrada(prompt, self.tokenizador, self.dispositivo)

        # 5. Generar la respuesta
        respuesta = generar_respuesta(self.modelo, entrada, self.tokenizador, prompt, parametros_generacion)

        # 6. Agregar respuesta al contexto
        self.gestor_contexto.agregar_mensaje("asistente", respuesta)

        # 7. Devolver la respuesta
        return respuesta

# === Personalización con PEFT/LoRA ===

def configurar_peft(modelo, r=8, lora_alpha=32):
    """
    Configura el modelo para fine-tuning con PEFT/LoRA.

    Args:
        modelo: Modelo base
        r (int): Rango de adaptadores LoRA
        lora_alpha (int): Escala alpha para LoRA

    Returns:
        modelo: Modelo adaptado para fine-tuning
    """
    # Identificar automáticamente los módulos de atención basados en la arquitectura del modelo
    target_modules = None

    # Detectar tipo de modelo basado en atributos
    if hasattr(modelo, "gpt_neox"):
        # Para modelos tipo GPT-NeoX
        target_modules = ["attention.query_key_value", "attention.dense"]
    elif hasattr(modelo, "transformer"):
        if hasattr(modelo.transformer, "h") and hasattr(modelo.transformer.h[0], "attn"):
            # Para modelos GPT-2
            target_modules = ["attn.c_attn", "attn.c_proj"]
        elif hasattr(modelo.transformer, "encoder") and hasattr(modelo.transformer.encoder, "layer"):
            # Para modelos tipo BERT/RoBERTa
            target_modules = ["attention.self.query", "attention.self.key", "attention.self.value", "attention.output.dense"]
    elif hasattr(modelo, "model") and hasattr(modelo.model, "decoder"):
        # Para modelos tipo T5
        target_modules = ["q", "v"]

    if not target_modules:
        print("No se pudo detectar automáticamente los módulos objetivo, usando configuración genérica")
        target_modules = ["query", "value", "key", "out_proj", "dense"]

    print(f"Configurando LoRA con target_modules: {target_modules}")

    lora_config = LoraConfig(
        r=r,
        lora_alpha=lora_alpha,
        target_modules=target_modules,
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM
    )

    modelo_peft = get_peft_model(modelo, lora_config)
    modelo_peft.print_trainable_parameters()

    return modelo_peft

def guardar_modelo(modelo, tokenizador, ruta):
    """
    Guarda el modelo y tokenizador en una ruta específica.

    Args:
        modelo: Modelo a guardar
        tokenizador: Tokenizador del modelo
        ruta (str): Ruta donde guardar
    """
    modelo.save_pretrained(ruta)
    tokenizador.save_pretrained(ruta)
    print(f"Modelo y tokenizador guardados en: {ruta}")

def cargar_modelo_personalizado(ruta):
    """
    Carga un modelo personalizado desde una ruta específica.

    Args:
        ruta (str): Ruta del modelo

    Returns:
        tuple: (modelo, tokenizador, dispositivo)
    """
    dispositivo = verificar_dispositivo()
    modelo = AutoModelForCausalLM.from_pretrained(ruta)
    tokenizador = AutoTokenizer.from_pretrained(ruta)

    # Configurar pad_token si no está definido
    if tokenizador.pad_token is None:
        tokenizador.pad_token = tokenizador.eos_token
        tokenizador.pad_token_id = tokenizador.eos_token_id

    modelo.to(dispositivo)
    return modelo, tokenizador, dispositivo

# === Interfaz web con Gradio ===

def crear_chatbot_con_memoria(modelo, tokenizador, dispositivo):
    """
    Crea una instancia de chatbot con memoria de conversación.

    Args:
        modelo: Modelo de lenguaje
        tokenizador: Tokenizador del modelo
        dispositivo: Dispositivo (CPU/GPU)

    Returns:
        Chatbot: Instancia del chatbot
    """
    instrucciones = "Eres un asistente virtual amable y servicial. Intenta dar respuestas útiles y detalladas a las preguntas del usuario."
    return Chatbot(modelo, tokenizador, dispositivo, instrucciones_sistema=instrucciones)

def crear_interfaz_web(chatbot, parametros_predefinidos=None):
    """
    Crea una interfaz web para el chatbot usando Gradio.

    Args:
        chatbot: Instancia del chatbot
        parametros_predefinidos: Parámetros de generación predefinidos

    Returns:
        gr.Interface: Interfaz de Gradio
    """
    # Estado para almacenar el historial de la conversación
    historial_chat = []

    def responder_mensaje(mensaje, historial=None, temperatura=0.7, top_p=0.9, max_tokens=100):
        if historial is None:
            historial = []

        # Configurar parámetros de generación personalizados
        parametros = {
            "max_new_tokens": max_tokens,
            "temperature": temperatura,
            "top_p": top_p,
            "do_sample": True,
            "pad_token_id": chatbot.tokenizador.pad_token_id,
        }

        # Obtener respuesta del chatbot
        respuesta = chatbot.responder(mensaje, parametros_generacion=parametros)

        # Actualizar historial
        historial.append((mensaje, respuesta))
        return "", historial

    with gr.Blocks(title="Chatbot Personalizado con Memoria") as interfaz:
        gr.Markdown("# 🤖 Chatbot Personalizado")
        gr.Markdown("Este chatbot utiliza un modelo de lenguaje personalizado con adaptadores LoRA y mantiene memoria de la conversación.")

        with gr.Row():
            with gr.Column(scale=4):
                chat_output = gr.Chatbot(height=400, label="Conversación")
                mensaje_input = gr.Textbox(placeholder="Escribe tu mensaje aquí...", label="Mensaje")
                enviar_btn = gr.Button("Enviar", variant="primary")

            with gr.Column(scale=1):
                gr.Markdown("### Configuración")
                temperatura_slider = gr.Slider(minimum=0.1, maximum=1.0, value=0.7, step=0.05, label="Temperatura")
                top_p_slider = gr.Slider(minimum=0.1, maximum=1.0, value=0.9, step=0.05, label="Top-p")
                max_tokens_slider = gr.Slider(minimum=10, maximum=200, value=100, step=10, label="Máx. tokens")
                clear_btn = gr.Button("Nueva conversación")

        # Eventos
        enviar_btn.click(
            fn=responder_mensaje,
            inputs=[mensaje_input, chat_output, temperatura_slider, top_p_slider, max_tokens_slider],
            outputs=[mensaje_input, chat_output]
        )
        mensaje_input.submit(
            fn=responder_mensaje,
            inputs=[mensaje_input, chat_output, temperatura_slider, top_p_slider, max_tokens_slider],
            outputs=[mensaje_input, chat_output]
        )
        clear_btn.click(lambda: ([], []), outputs=[mensaje_input, chat_output])

    return interfaz

# === Despliegue ===

def main_despliegue(modelo_base="Qwen/Qwen2.5-0.5B-Instruct", usar_modelo_personalizado=False, ruta_modelo_personalizado="modelo_personalizado"):
    """
    Función principal para el despliegue del chatbot.

    Args:
        modelo_base (str): Identificador del modelo base en HuggingFace
        usar_modelo_personalizado (bool): Si True, carga un modelo personalizado guardado
        ruta_modelo_personalizado (str): Ruta donde se encuentra el modelo personalizado
    """
    if usar_modelo_personalizado:
        print(f"Cargando modelo personalizado desde: {ruta_modelo_personalizado}")
        modelo, tokenizador, dispositivo = cargar_modelo_personalizado(ruta_modelo_personalizado)
    else:
        print(f"Cargando modelo base: {modelo_base}")
        modelo, tokenizador, dispositivo = cargar_modelo(modelo_base)

        # Como opción: Aplicar PEFT/LoRA al modelo base
        aplicar_peft = False
        if aplicar_peft:
            print("Aplicando adaptadores LoRA al modelo base...")
            modelo = configurar_peft(modelo)

            # Guardar el modelo personalizado
            guardar_modelo(modelo, tokenizador, ruta_modelo_personalizado)
            print(f"Modelo personalizado guardado en: {ruta_modelo_personalizado}")

    # Crear instancia del chatbot
    chatbot = crear_chatbot_con_memoria(modelo, tokenizador, dispositivo)

    # Crear y lanzar la interfaz web
    print("Iniciando la interfaz web...")
    interfaz = crear_interfaz_web(chatbot)
    interfaz.launch(share=True)  # share=True permite acceso público a través de internet
    print("Interfaz web iniciada.")

if __name__ == "__main__":
    # Opciones de configuración
    options = {
        "modelo_base": "Qwen/Qwen2.5-0.5B-Instruct",  # Modelo base a utilizar
        "usar_modelo_personalizado": False,  # Cambiar a True para cargar un modelo personalizado
        "ruta_modelo_personalizado": "modelo_personalizado"  # Ruta del modelo personalizado
    }

    main_despliegue(**options)

Cargando modelo base: Qwen/Qwen2.5-0.5B-Instruct
Utilizando dispositivo: cpu
Iniciando la interfaz web...


<ipython-input-16-e2c36589aaad>:320: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chat_output = gr.Chatbot(height=400, label="Conversación")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f53388deaee8008ece.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Interfaz web iniciada.


# ***Preguntas Teóricas***

*¿Cuáles son las diferencias fundamentales entre los modelos encoder-only, decoder-only y encoder-decoder en el contexto de los chatbots conversacionales? Explique qué tipo de modelo sería más adecuado para cada caso de uso y por qué.*

## **Respuesta:**

Basicamente, los modelos encoder-only, (como por ejemplo el modelo BERT), se enfocan en entender texto, por lo que son ideales para tareas tales como de clasificación, detección de intención o análisis de sentimientos, pero no generan respuestas. Por otro lado, los decoder-only, tales como GPT, generan texto de forma autoregresiva y son los más adecuados para chatbots conversacionales abiertos, ya que pueden producir respuestas coherentes y lo más importante, es que lo hacen turno tras turno. Finalmente, los modelos encoder-decoder, (así como T5 o BART), combinan la comprensión y generación, siendo útiles para tareas tan importantes como traducción, resumen o respuestas basadas en un contexto cerrado. Esto lo que nos permite concluir, es que para chatbots de conversación abierta y continua, los decoder-only son una buena opción.

*Explique el concepto de "temperatura" en la generación de texto con LLMs. ¿Cómo afecta al comportamiento del chatbot y qué consideraciones debemos tener al ajustar este parámetro para diferentes aplicaciones?*

## **Respuesta:**

Como pudimos comprobar, la temperatura es un parámetro que regula la aleatoriedad en la generación de texto. Una temperatura baja (como por ejemplo una de 0.3) hace que el modelo sea más preciso y repetitivo, ideal para tareas donde se requiere confiabilidad y exactitud. Por el contrario, una temperatura alta (como por ejemplo una de 1.0) promueve en el bot más diversidad y creatividad, pero hay que tener en cuenta que esto puede llevar a errores o incoherencias. Así que, el ajustar este valor depende del uso y del contexto de desarrollo: en asistentes técnicos o educativos conviene usar temperaturas bajas, mientras que en aplicaciones creativas o de entretenimiento seria preferible el usar unas temperaturas más altas.



*Describa las técnicas principales para reducir el problema de "alucinaciones" en chatbots basados en LLMs. ¿Qué estrategias podemos implementar a nivel de inferencia y a nivel de prompt engineering para mejorar la precisión factual de las respuestas?*

## **Respuesta:**

Para mitigar alucinaciones (o tambien conocidas como respuestas incorrectas), se pueden aplicar técnicas tanto durante la inferencia como en el diseño del prompt. A nivel de inferencia, usar una temperatura baja, tales estrategias como el top-k/top-p sampling y modelos con recuperación de información externa (como por ejemplo RAG) ayuda a mejorar la precisión. Debemos tambien tener en cuanta que en cuanto al prompt engineering, dar instrucciones explícitas, incluir contexto relevante y utilizar ejemplos guía (que en ingles a esto se le conoce como few-shot prompting) nos puede ayudar a reducir errores. En ultima instancia, también es útil pedirle al modelo que diga “no sé” si no tiene certeza, en lugar de inventar una respuesta o simplemente, como se le llama a una "alucinación".

HECHO POR: Julian David Navarro G.

IA - Mayo de 2025